In [23]:
import pandas as pd
import xarray as xr
import numpy as np
import netCDF4 as nc
#from scipy.spatial import distance

In [24]:
# Open catalogue
df = pd.read_csv('/home/data/ReAnalysis/ERA5/Storm_analysis/NAECv1/NAEC_1979_2020_v1.csv')

# open netcdf mask file
file = '/pampa/picart/Masks/mask_GEM5_ERA5grid'
data = xr.open_dataset(file)

# export netcdf to dataframe and drop index
mk = data.to_dataframe().reset_index()

# Only keep grid points coordinates that are within CRCM6 domain
mk = mk.loc[mk.HU == True]

# open crcm6 boundary layer 
bnd = pd.read_csv('/pampa/cloutier/outline_crcm6_domain.csv', index_col = 0)

In [34]:
# Chen filtre les ETC qui sont pendant au moins 24h CONSÉCUTIVES dans le domaine et dont le centre est
# à une distance minimale de 5° du bord du domaine de CRCM6.

# on doit donc déterminer une fonction qui calcule la distance entre un point de grille et le bord du 
# domaine de CRCM6

# Il faut que le point de grille ait une valeur HU == True et qu'il soit è une distance minimale de 5° de 
# tous les points de grille qui tracent la limite 
mk = mk.rename(columns={'lat' : 'latitude', 'lon' : 'longitude'})
merge = df.merge(mk, how='left', on=['latitude', 'longitude'])
merge = merge.fillna(value = False)
df24_consec = pd.DataFrame(columns = df.columns)

In [37]:
# Fonction qui détermine si le point donné dans le catalogue est à une distance 
# minimale de tous les points de grille du frame du domaine de crcm6 de 5°

def get_distance(latS, lonS, bnd) :
    dist_cond = True

    for _, row2 in bnd.iterrows():
        latD = row2['lat']
        lonD = row2['lon']
        dist = ((latS-latD)**2 + (lonS - lonD)**2)**0.5
        
        if dist < 5 : 
            dist_min = False
            break
            
    return dist_cond

In [38]:
# Iterate through each storm
for storm_id in merge['storm'].unique():
    storm_data = merge[merge['storm'] == storm_id].copy()
    count = 0
   
    # Iterate through each grid point
    for _, row in storm_data.iterrows() :
        dist_cond = False
        hu = row['HU']
        latS = row['latitude']
        lonS = row['longitude']
        
        # If the grid point is in the subdomain 
        if hu : 
            dist_cond = get_distance(latS, lonS, bnd)
    
        # No need to put and HU == True in the next if condition, because if dist_cond = true, 
        # it means that the if hu conditions was respected (dist_cond is reset to false for each 
        # grid point iteration )
        if dist_cond : 
            count += 1

        if hu == False and count >= 24 :
            break

        if hu == False and count < 24 :
            count = 0

    if count >= 24 : 
        df24_consec = df24_consec.append(storm_data)
        print('Year in process : ', df24_consec['datetime'].iloc[-1])

Year in process :  1979010204
Year in process :  1979010218
Year in process :  1979010822
Year in process :  1979011015
Year in process :  1979011100
Year in process :  1979011818
Year in process :  1979012105
Year in process :  1979012315
Year in process :  1979012421
Year in process :  1979012821
Year in process :  1979020119
Year in process :  1979020309
Year in process :  1979020706
Year in process :  1979020712
Year in process :  1979020908
Year in process :  1979021007
Year in process :  1979021211
Year in process :  1979022318
Year in process :  1979022406
Year in process :  1979022709
Year in process :  1979030110
Year in process :  1979030217
Year in process :  1979030315
Year in process :  1979030509
Year in process :  1979030715
Year in process :  1979030906
Year in process :  1979031203
Year in process :  1979031523
Year in process :  1979031706
Year in process :  1979032121
Year in process :  1979032518
Year in process :  1979033000
Year in process :  1979033017
Year in pr

KeyboardInterrupt: 

In [31]:

        mk_filtered = mk.loc[(mk['latitude'] <= latS - 5) | (mk['latitude'] >= latS + 5) |
                             (mk['longitude'] <= lonS - 5) | (mk['longitude'] >= lonS + 5)]

28

In [42]:
def test(x,y) : 
    c = x
    d = y

    return c, d

a = 3
b = 4

u, v = test(a,b)

print(u, v)

3 4
